<a href="https://colab.research.google.com/github/RachelZhou287/542_Final/blob/main/BNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torchvision
print(torchvision.__version__)
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import os


0.23.0+cu126


In [ ]:



# ============================================================
# 1. Download CUB-200-2011 if missing
# ============================================================
data_root = "/content/data/CUB_200_2011"
if not os.path.exists(data_root):
    os.makedirs("/content/data", exist_ok=True)
    url = "http://www.vision.caltech.edu/visipedia-data/CUB-200-2011/CUB_200_2011.tgz"
    print("Downloading CUB_200_2011 (~1.1GB)...")
    !wget -q $url -O /content/data/CUB_200_2011.tgz
    !tar -xzf /content/data/CUB_200_2011.tgz -C /content/data/
    print("✅ Download complete and extracted.")

# ============================================================
# 2. Define custom dataset
# ============================================================
class CUBDataset(Dataset):
    def __init__(self, root, train=True, transform=None):
        self.root = root
        self.transform = transform

        # metadata
        img_files = pd.read_csv(os.path.join(root, "images.txt"), sep=" ", names=["img_id", "filepath"])
        labels = pd.read_csv(os.path.join(root, "image_class_labels.txt"), sep=" ", names=["img_id", "target"])
        split = pd.read_csv(os.path.join(root, "train_test_split.txt"), sep=" ", names=["img_id", "is_training_img"])

        df = img_files.merge(labels, on="img_id").merge(split, on="img_id")
        df = df[df["is_training_img"] == int(train)]
        self.paths = df["filepath"].values
        self.targets = df["target"].values - 1

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images", self.paths[idx])
        img = Image.open(img_path).convert("RGB")
        label = self.targets[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# ============================================================
# 3. Transforms and loaders
# ============================================================
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = CUBDataset(data_root, train=True, transform=transform)
test_dataset  = CUBDataset(data_root, train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"✅ Loaded {len(train_dataset)} train and {len(test_dataset)} test images")

# ============================================================
# 4. Visualize a few samples
# ============================================================
imgs, labels = next(iter(train_loader))
plt.figure(figsize=(10, 4))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(imgs[i].permute(1, 2, 0))
    plt.title(f"Class {labels[i].item()}")
    plt.axis("off")
plt.show()


In [ ]:
!rm -f /content/CUB_200_2011.tgz


In [ ]:
!mkdir -p /content/data
!wget -q http://www.vision.caltech.edu/visipedia-data/CUB-200-2011/CUB_200_2011.tgz -O /content/data/CUB_200_2011.tgz
!tar -xzf /content/data/CUB_200_2011.tgz -C /content/data/



gzip: stdin: unexpected end of file
tar: Child returned status 1
tar: Error is not recoverable: exiting now
